In [155]:
import dlib
import cv2
from imutils import face_utils
from scipy.spatial import distance as dist
from threading import Thread
import numpy as np
import time

In [156]:
def eye_aspect_ratio(eye):
    dist1=dist.euclidean(eye[0],eye[5])
    dist2=dist.euclidean(eye[2],eye[4])
    dist3=dist.euclidean(eye[0],eye[3])
    ear=(dist1+dist2)/dist3
    return ear

In [157]:
def eye(eye):
    (right_eye1,right_eye2)=face_utils.FACIAL_LANDMARKS_IDXS['right_eye']
    (left_eye1,left_eye2)=face_utils.FACIAL_LANDMARKS_IDXS['left_eye']
    right_eye=eye[right_eye1:right_eye2]
    right_ear=eye_aspect_ratio(right_eye)
    left_eye=eye[left_eye1:left_eye2]
    left_ear=eye_aspect_ratio(left_eye)
    total_ear=(right_ear+left_ear)/2
    return (total_ear,right_eye,left_eye)

In [158]:
def lip_dist(shape):
    (inner1,inner2)=face_utils.FACIAL_LANDMARKS_IDXS['inner_mouth']
    (outer1,outer2)=face_utils.FACIAL_LANDMARKS_IDXS['mouth']
    upper_lips=shape[61:65]
    lower_lips=shape[65:68]
    upper_corr=[]
    lower_corr=[]
    for i in upper_lips:
        upper_corr.append(i[1])
    for i in lower_lips:
        lower_corr.append(i[1])
    upper_lip_mean=np.mean(upper_corr)
    lower_lip_mean=np.mean(lower_corr)
    diff=abs(upper_lip_mean-lower_lip_mean)
    return diff

In [159]:
cap=cv2.VideoCapture(0)
detector=dlib.get_frontal_face_detector()
predictor=dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [160]:
count=0
count1=0
while True:
    ret,frame=cap.read()
    grey=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    face=detector(grey)
    for i in face:
        landmark=predictor(grey,i)
        shape=face_utils.shape_to_np(landmark)
        ear,right_eye,left_eye=eye(shape)
        lip_dis=lip_dist(shape)
        if ear<0.6:
            if count==0:
                start_time=time.time()
            count+=1
            stop_time=time.time()
            if abs(start_time-stop_time)>4:
                cv2.putText(frame,'Drowsiness alert!!!',(30,50),cv2.FONT_ITALIC,0.5,(0,255,0),1)
        else:
            count=0
        #cv2.putText(frame,'Drowsiness alert!!!'+str(abs(start_time-stop_time)),(30,50),cv2.FONT_ITALIC,0.5,(0,255,0),1)
        if lip_dis>12:
            if count1==0:
                start_time1=time.time()
            stop_time1=time.time()
            count1+=1
            if abs(start_time1-stop_time1)>2:
                cv2.putText(frame,'Yawn alert!!!',(50,70),cv2.FONT_ITALIC,0.5,(0,255,0),2)
        else:
            count1=0
        #cv2.putText(frame,'Yawn alert!!!'+str(abs(start_time1)),(50,70),cv2.FONT_ITALIC,0.5,(0,255,0),1)
        lip_corr=shape[60:68]
        right_eye_hull=cv2.convexHull(right_eye)
        left_eye_hull=cv2.convexHull(left_eye)
        cv2.drawContours(frame,[right_eye_hull],-1,(0,255,0),2)
        cv2.drawContours(frame,[left_eye_hull],-1,(0,255,0),2)
        cv2.drawContours(frame,[lip_corr],-1,(0,255,255),2)
        cv2.putText(frame,f'Ear:{ear} and lip dist:{lip_dis}',(10,30),cv2.FONT_ITALIC,0.5,(0,255,0),1)
        cv2.imshow('output',frame)
    if cv2.waitKey(10)==13:
        break
cap.release()
cv2.destroyAllWindows()